In [4]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
Gold_lakehouse_Path = "abfss://BARQ_Intenrship@onelake.dfs.fabric.microsoft.com/Gold_lakehouse.Lakehouse/Tables"

df = spark.read.parquet("Files/sales_data/sales_cleaned")


# Load dimensions
dim_customer = spark.read.format("delta").load(f"{Gold_lakehouse_Path}/DimCustomer")
dim_product = spark.read.format("delta").load(f"{Gold_lakehouse_Path}/DimProduct")
dim_store = spark.read.format("delta").load(f"{Gold_lakehouse_Path}/DimStore")



StatementMeta(, 7ab55832-a700-4179-bad5-8044a89fb3b3, 6, Finished, Available, Finished)

StatementMeta(, 7ab55832-a700-4179-bad5-8044a89fb3b3, 7, Finished, Available, Finished)

+----------+----------------+-----+------------+----------+-----------+---------+--------+------------+----------+---------------+--------------+------------+
|store_city|product_category|color|       brand|unit_price| first_name|last_name|quantity|total_amount| sale_date|Dim_Customer_ID|Dim_Product_ID|Dim_Store_ID|
+----------+----------------+-----+------------+----------+-----------+---------+--------+------------+----------+---------------+--------------+------------+
|  Hurghada|          Hoodie|Black|        Nike|     225.0|     Joshua|     Clay|       3|       675.0|2022-04-04|           4568|          1428|           5|
|  Hurghada|         T-Shirt|White|        Nike|     144.0|   Jennifer|   Hansen|       5|       720.0|2022-02-19|           3883|          9222|           5|
|     Cairo|        Sneakers| Blue|        Nike|     211.0|Christopher|     Bell|       1|       211.0|2022-02-24|           1642|          5603|           3|
|     Cairo|         T-Shirt|Green|      Reebo

In [8]:
df_fact = df_fact.select("Dim_Customer_ID", "Dim_Product_ID", "Dim_Store_ID",
                "quantity", "total_amount", "sale_date")

df_fact.show()

StatementMeta(, 7ab55832-a700-4179-bad5-8044a89fb3b3, 10, Finished, Available, Finished)

+---------------+--------------+------------+--------+------------+----------+
|Dim_Customer_ID|Dim_Product_ID|Dim_Store_ID|quantity|total_amount| sale_date|
+---------------+--------------+------------+--------+------------+----------+
|           4568|          1428|           5|       3|       675.0|2022-04-04|
|           3883|          9222|           5|       5|       720.0|2022-02-19|
|           1642|          5603|           3|       1|       211.0|2022-02-24|
|            330|          8877|           3|       5|      1005.0|2025-04-02|
|           4739|          2002|           6|       2|       658.0|2022-08-28|
|           4110|          6283|           1|       4|      1096.0|2022-02-08|
|           2726|          5111|           4|       1|       147.0|2023-01-01|
|           4579|           110|           4|       5|      1175.0|2025-04-26|
|           9121|          6877|           2|       1|       293.0|2025-01-29|
|           2880|          3496|           5|       

In [11]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
df_fact = df_fact.withColumn("Fact_ID",row_number().over(Window.orderBy("sale_date")))



StatementMeta(, 7ab55832-a700-4179-bad5-8044a89fb3b3, 13, Finished, Available, Finished)

In [12]:
df_fact.show()

StatementMeta(, 7ab55832-a700-4179-bad5-8044a89fb3b3, 14, Finished, Available, Finished)

+---------------+--------------+------------+--------+------------+----------+-------+
|Dim_Customer_ID|Dim_Product_ID|Dim_Store_ID|quantity|total_amount| sale_date|Fact_ID|
+---------------+--------------+------------+--------+------------+----------+-------+
|           3001|          5815|           3|       3|      1485.0|2022-01-01|      1|
|           2463|            20|           1|       2|       536.0|2022-01-01|      2|
|           5680|          7607|           2|       5|      1750.0|2022-01-01|      3|
|           3055|          9267|           5|       1|       185.0|2022-01-01|      4|
|            978|          8388|           1|       4|       708.0|2022-01-01|      5|
|           2964|          3793|           3|       4|       772.0|2022-01-01|      6|
|           3134|          1089|           3|       5|       705.0|2022-01-01|      7|
|           4141|          6982|           5|       2|       544.0|2022-01-02|      8|
|           1822|          2862|           

In [13]:
df_fact.write.mode("overwrite").format("delta").save(f"{Gold_lakehouse_Path}/FactSales")

StatementMeta(, 7ab55832-a700-4179-bad5-8044a89fb3b3, 15, Finished, Available, Finished)